# 第十六章 类和函数
## 16.1 时间
编写一个print_time的函数，接收一个Time对象作为形参并以“hour：minute：second”的格式打印。<br>
提示：格式序列'%.2d'可以以最少两个字符打印一个整数，如果需要，会在前面添加前缀0。<br>

上述提示用的是旧式字符串格式化方法，应采用新的两种字符串格式方法：**1.f-string 2.str.format**<br>
旧式字符串格式化方法 ：https://docs.python.org/zh-cn/3/library/stdtypes.html#printf-style-string-formatting
1. f-string：https://docs.python.org/zh-cn/3/reference/lexical_analysis.html#f-strings
2. str.format:https://docs.python.org/zh-cn/3/library/stdtypes.html#str.format

In [100]:
class Time:
    def __init__(self,hour,minute,second):
        self.hour =  hour
        self.minute = minute
        self.second = second
        
    def print_time(self):
        print('{:02}:{:02}:{:02}'.format(self.hour,self.minute,self.second))
        
    #     比较时间
    def is_after(self,other):
        return time_to_int(self) > time_to_int(other)
    
    

In [101]:
time1 = Time(11,1,1)
time1.print_time()

11:01:01


In [102]:
time2 = Time(10,10,10)
time2.print_time()

10:10:10


编写一个布尔函数is_after，接收两个Time对象。t1和t2，若t1在t2时间之后返则返回True，否则返回False。挑战：不用if表达式

In [103]:
time1.is_after(time2)

True

## 16.2 纯函数
编写两个用来增加时间值的函数。<br>
展示两种不同类型的函数：**纯函数**和**修改器**。<br>
展示称为**原型和补丁（prototype and patch）**的开发计划。

下面是add_time的一个简单原型:<br>
这个函数创建一个新的Time对象，并返回这个新对象的一个引用。这被称为一个**纯函数**。<br>
除了返回一个值之外，并不修改作为实参传入的任何对象，也没有任何如显示值或获得用户输入之类的副作用。

In [104]:
def add_time(t1,t2):
    sum = Time(t1.hour+t2.hour , t1.minute+t2.minute , t1.second+t2.second)
    return sum

创建两个time对象：<br>
start存放一个电影的开始时间<br>
duration存放电影的播放时间<br>

add_time计算出电影何时结束

In [105]:
start = Time(9,45,0)
duration = Time(1,35,0)
done = add_time(start,duration)
done.print_time()

10:80:00


结果10:80:00中分钟超过了60，所以我们需要改进add_time函数。

In [106]:
def add_time(t1,t2):
    sum = Time(0,0,0)
    
    sum.second = t1.second + t2.second
    sum.minute = t1.minute + t2.minute 
    sum.hour = t1.hour + t2.hour 
    
    if sum.second >= 60:
        sum.second-=60
        sum.miunte+=1
    if sum.minute >= 60:
        sum.minute-=60
        sum.hour+=1
        
    return sum

虽然这个函数是正确的，它已经开始变大了。我们会在后面看到一个更短的版本。

In [107]:
start = Time(9,45,0)
duration = Time(1,35,0)
done = add_time(start,duration)
done.print_time()

11:20:00


## 16.3 修改器
修改传入的参数对象，修改对调用者可见，这样的函数称为**修改器（modifier）**。<br>    
函数increment给一个Time对象增加指定的描述，下面是一个初稿：

In [108]:
def increment(time,seconds):
    time.second += seconds
    
    if time.second >= 60:
        time.second -= 60
        time.minute += 1
    if time.minute >= 60:
        time.minute -= 60
        time.hour += 1

如果seconds比60大很多，比如下例中seconds为800，就会发现只进一次位是不够的；需要重复进位，一种办法是使用while语句代替if语句，那样会让函数正确，但不是很高效。

In [109]:
time2 = Time(20,20,20)
increment(time2,800)
time2.print_time()

20:21:760


作为练习，编写正确的increment版本，不包括任何循环。

In [110]:
def increment(time,seconds):
    time.second += seconds
    
    time.minute += int(time.second/60)
    time.second = time.second%60

    time.hour += int(time.minute/60)
    time.minute = time.minute%60

In [111]:
time2 = Time(20,59,20)
time2.print_time()
increment(time2,800)
time2.print_time()

20:59:20
21:12:40


任何可以使用修改器做到的功能都可以使用纯函数实现，纯函数的程序比修改器的程序开发更快，错误更少。<br>
尽量编写纯函数，只在有绝对说服力原因的时候才使用修改器。这种方法被称为**函数式编程风格**。<br>
作为练习，编写一个increment的纯函数版本，创建并返回一个新的Time对象，而不修改参数。

In [112]:
def increment(time,seconds):
    modified_time = Time(time.hour,time.minute,time.second)
    
    modified_time.second += seconds
    
    modified_time.minute += int(modified_time.second/60)
    modified_time.second = modified_time.second%60

    modified_time.hour += int(modified_time.minute/60)
    modified_time.minute = modified_time.minute%60
    
    return modified_time

In [113]:
time2 = Time(20,59,20)
time2.print_time()
time3 = increment(time2,800)
time3.print_time()

20:59:20
21:12:40


## 16.4 原型和计划
刚才我们展示的开发计划称为“原型和补丁”。对每个函数，我可以先编写一个基本的原型，再不断的进行测试发现错误，并打补丁改进。<br>
另一种方法是**有规划开发（designed development）**。比如上面的问题中，Time对象实际上是六十进制数里的三位数。<br>
在编写add_time和increment时，我们实际上是在六十进制上进行加减，因此才需要从一位加到另一位。那我们可以将Time对象转换成整数。<br>

下面是一个将Time对象转换成整数的函数：

In [114]:
def time_to_int(time):
    minutes = time.minute + time.hour * 60
    seconds = time.second + minutes * 60
    return seconds

下面是一个将整数转换回Time对象的函数

In [115]:
def int_to_time(seconds):
    time = Time(0,0,0)
    minutes ,time.second = divmod(seconds , 60)
    time.hour , time.minute = divmod(minutes , 60)
    return time

那么怎么测试上面两个函数呢，一种测试的方法可以是用很多x值检查`time_to_int(int_to_time(x)) == x`

In [116]:
x = 34567
time = int_to_time(x)
time.print_time()

09:36:07


In [117]:
time = Time(9,36,7)
x = time_to_int(time)
x

34567

一旦认为他们是正确的，就可以使用他们重写add_time:

In [118]:
def add_time(t1,t2):
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

作为练习，重写increment函数。

In [119]:
def increment(t,seconds):
    seconds += time_to_int(t)
    return int_to_time(seconds)

## 16.5 调试
一个Time对象的minute和second的值应当在0与60之间，以及hour是正值，hour和minute应当是整数值，也允许second拥有小数值。<br>
这些需求称为**不变式**，以为它们应当总为真，所以可编写一个像valid_time这样的函数，接收Time对象来帮助你检查不变式，当违反不变式时，返回Fasle。

In [120]:
def valid_time(time):
    if not isinstance(time.hour,int) or not isinstance(time.minute,int):
        return False
    if time.hour < 0 or time.minute < 0 or time.second <0:
        return False
    if time.minute >= 60 or time.second >= 60:
        return False
    return True

这样我们就可以使用一个assert语句检查一个给定的不定式，当检查失败时抛出异常:

In [121]:
def add_time(t1,t2):
    assert valid_time(t1) and valid_time(t2)
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

In [124]:
start = Time(2,2,2)
during = Time(3,3.1,3)
done = add_time(start,during)
done.print_time()

AssertionError: 

## 16.6 术语表
- 原型和补丁（prototype and patch）:一种开发计划模式，先编写程序的粗略原型，并测试。在找到错误时改正。
- 有规划开发（planned development）：一种开发计划模式，先对问题有了高阶的深入理解，并且比增量开发或者原型开发有更多的规划。
- 纯函数（pure function）：不修改任何形式参数对象的函数。大部分纯函数都有返回值。
- 修改器（modifier）：修改一个或者多个形参对象的函数。大部分修改器都不返回值，或者返回None。
- 函数式编程风格（functional programming style）：一种编程设计风格，其中大部分函数都是纯函数。
- 不变式（invariant）：在程序的执行过程中应当总是为真的条件。
- assert 语句（assert statement）：一种检查某个条件，如果检查失败则会抛出异常的语句。

## 16.7 练习

## 16.8 自我总结
1. 字符串格式化方法，应采用新的两种字符串格式方法：`1.f-string 2.str.format`
2. 学习了两种开发计划模式，分别是**原型和补丁，有规划开发**
3. 学习了内置函数`divmod()`的使用方法
4. 学习了`assert`语句的简单用法
5. 编写函数时尽量使用纯函数，少用修改器。